In [7]:
import numpy as np
import pandas as pd
import pyddm
import os
import utils
import models

In [14]:
def get_model_measures(model, condition):
    # print(condition)
    sol = model.solve(condition)
    mean_rt_error = np.sum(sol.pdf_err()*model.t_domain())*model.dt / sol.prob_error()
    # return condition["tta_0"], condition["d_0"], condition["a_1"], condition["a_2"], \
    #        sol.prob_correct(), sol.mean_decision_time(), mean_rt_error
    return condition["tta_0"], condition["d_0"], condition["a_values"], condition["a_duration"],\
           sol.prob_correct(), sol.mean_decision_time(), mean_rt_error

def get_model_rt_distr(model, condition, kind="cdf"):
    sol = model.solve(condition)
    return pd.DataFrame({"tta_0": condition["tta_0"],
                         "d_0": condition["d_0"],
                         "a_values": condition["a_values"],
                         "a_duration": condition["a_duration"],
                         # "a_1": condition["a_1"],
                         # "a_2": condition["a_2"],
                         "t": model.t_domain(),
                         "rt_corr_distr": (sol.cdf_corr() if kind=="cdf" else sol.pdf_corr())/sol.prob_correct(),
                         "rt_error_distr": (sol.cdf_err() if kind=="cdf" else sol.pdf_err())/sol.prob_error()})

def initialize_model(model_no, param_set):
    overlay = models.OverlayNonDecisionGaussian(ndt_location=param_set.ndt_location, ndt_scale=param_set.ndt_scale)
    bound = models.BoundCollapsingTta(b_0=param_set.b_0, k=param_set.k, tta_crit=param_set.tta_crit)

    if model_no == 1:
        # drift only depends on TTA and distance (2020 version)
        drift = models.DriftTtaDistance(alpha=param_set.alpha, beta_d=param_set.beta_d, theta=param_set.theta)
    if model_no == 2:
        # same as model 1 but dynamic calculation of tta and d
        drift = models.DriftFixedAcceleration(alpha=param_set.alpha, beta_d=param_set.beta_d, theta=param_set.theta)
    elif model_no == 3:
        # acceleration-dependent drift; acceleration condition implemented as a vector
        drift = models.DriftAccelerationDependent(alpha=param_set.alpha, beta_d=param_set.beta_d, beta_a=param_set.beta_a, theta=param_set.theta)
    elif model_no == 4:
        # acceleration-dependent drift; acceleration condition implemented as a vector
        drift = models.DriftAccelerationDependent_v2(alpha=param_set.alpha, beta_d=param_set.beta_d,
                                                     beta_a=param_set.beta_a, theta=param_set.theta)
    else:
        model = None

    model = pyddm.Model(name="Model %i" % model_no, drift=drift, bound=bound, overlay=overlay,
                      noise=pyddm.NoiseConstant(noise=1), T_dur=models.ModelTtaDistance.T_dur)

    return model

def simulate_model(model_no, param_set, conditions, ret="measures"):
    """
    Set ret to "measures" or "rt_cdf" or "rt_pdf" for saving p_turn and mean RT or RT CDF or RT PDF
    """
    model = initialize_model(model_no, param_set)

    if ret=="measures":
        sim_result = pd.DataFrame([get_model_measures(model, condition) for condition in conditions],
                                  # columns=["tta_0", "d_0", "a_1", "a_2", "is_gap_accepted", "RT_go", "RT_stay"])
                                  columns=["tta_0", "d_0", "a_values", "a_duration", "is_gap_accepted", "RT_go", "RT_stay"])
    else:
        sim_result = pd.concat([get_model_rt_distr(model, condition, kind=ret[-3:]) for condition in conditions])
    sim_result["subj_id"] = param_set.subj_id
    return sim_result

In [15]:
def save_sim_results(model_no, file_name, conditions=None, cross_validation=False, ret="measures", prefix=""):
    if model_no == 1:
        model_name = "drift_tta_distance"
    elif model_no == 2:
        model_name = "drift_tta_distance_interp"
    elif model_no == 3:
        model_name = "model_acceleration_dependent"
    elif model_no == 4:
        model_name = "model_acceleration_dependent_v2"
    else:
        raise ValueError("Wrong model_no")

    if cross_validation:
        model_name += "_cross_validation"

    path = os.path.join("modeling/fit_results_replaced_nan_rt", model_name)
    parameters = pd.read_csv(os.path.join(path, file_name))

    sim_results = [simulate_model(model_no, param_set, conditions, ret=ret)
                   for idx, param_set in parameters.iterrows()]

    sim_results = pd.concat(sim_results)
    sim_results.to_csv(os.path.join(path, (prefix+file_name).replace("parameters_fitted", "sim_" + ret)), index=False)

In [16]:
conditions = [{"tta_0": tta_0, "d_0": d_0, "a_values": a_values, "a_duration": a_duration}
              for tta_0 in np.linspace(4, 6, 9)
              for d_0 in np.linspace(70, 90, 3)
              for a_values in [(0., 0., 0., 0.),
                               (0., 4, 4, 0.),
                               (0., 4, -4, 0.),
                               (0., -4, 4, 0.),
                               (0., -4, -4, 0.)]
              for a_duration in [1.0]]

# measures

In [6]:
save_sim_results(model_no=1, file_name="subj_all_parameters_fitted.csv", conditions=conditions, cross_validation=False,
                 ret="measures")

In [14]:
save_sim_results(model_no=2, file_name="subj_all_parameters_fitted.csv", conditions=conditions, cross_validation=False,
                 ret="measures")

In [17]:
save_sim_results(model_no=3, file_name="subj_all_parameters_fitted.csv", conditions=conditions, cross_validation=False,
                 ret="measures")

In [18]:
save_sim_results(model_no=3, file_name="subj_all_parameters_fitted.csv", conditions=conditions, cross_validation=True,
                 ret="measures")

# Predictions

In [5]:
a_durations = np.linspace(0.1, 3.0, 11)
a_magnitudes = np.linspace(0.0, 5.0, 11)

conditions = [{"tta_0": 6, "d_0": 80, "a_values": (0.0, -a_magnitude, a_magnitude, 0.0), "a_duration": a_duration}
              for a_duration in a_durations
              for a_magnitude in a_magnitudes]

In [6]:
save_sim_results(model_no=3, file_name="subj_all_parameters_fitted.csv", conditions=conditions, cross_validation=False,
                 ret="measures", prefix="prediction_")